# Goal

This notebook outputs a list of MR paths that are filtered: between 100-300 slices.

Available MR data from:
- ABIDE.txt (1160 files)
- ABVIB.txt (778 files)
- ACRIN-FMISO-Brain.txt (1403 files)
- ADNI.txt (2640 files)
- PPMI.txt (1524 files)

In [1]:
import os
import time
import pickle
from pathlib import Path

import SimpleITK as sitk

import numpy as np
from pandas import DataFrame as DF

from helpers_general import sitk2np, print_sitk_info, round_tuple, lrange, lmap, get_roi_range, numbers2groups

In [12]:
# wsl: /home/rgologorsky/DeepPit
hd_path = "../" * 5 + "/media/labcomputer/e33f6fe0-5ede-4be4-b1f2-5168b7903c7a" + "/home/"

# all folders in HD
all_folders = os.listdir(hd_path)

print(all_folders)

['linuxbrew', 'todd', 'lab_member']


In [3]:
# folders with data
data_folders = ["ABIDE", "ABVIB", "ACRIN-FMISO-Brain", "ADNI", "PPMI"]

# get paths to MR
data_folder_paths = []

for folder in data_folders:
    with open(f"{folder}.txt", "rb") as fp:   # Unpickling
        data_folder_paths.append(pickle.load(fp))

# Metadata

folder	imputedSeq	fn	sz	px	sp	dir

In [ ]:
# get extension
# assume all files in dir have same extension
def get_ext(dir_path):
    # assume all files in dir have same extension
    file = next(os.walk(dir_path))[2][0]
    
    # in case file is bytes not str
    try:
        file = file.decode()
    except:
        pass
    
    return Path(file).suffix.lower()

In [8]:
idx    = 0
folder = data_folders[idx]
files  = data_folder_paths[idx]

print(f"Folder = {folder}; No. files = {len(files)}")

Folder = ABIDE; No. files = 1160


In [9]:
files[:2]

['../../../mnt/d/PitMRdata/ABIDE/PAD/PAD_0001/T1-FFE/1995-03-11_00_00_00.0/S7478',
 '../../../mnt/d/PitMRdata/ABIDE/PAD/PAD_0002/T1-FFE/1995-04-04_00_00_00.0/S7479']

In [58]:
# data frame w/ meta data info
d = []

for path in enumerate(flat_train_paths):
    
    # get folder name = data src
    folder = get_folder_name(path)
    
    # get file ext (nii, dcm, etc)
    idx    = src2idx[folder]
    ext    = get_ext(path)
    
    # get file
    # ASSUMES only 1 nii in folder
    
    if ext == ".nii" or ext == ".dcm":
        file = os.listdir(str(path))[0]
        file = f"{path}/{file}"
    else:
        print(f"Weird ext - {ext}.")
    
    # read meta data
    reader = sitk.ImageFileReader()
    reader.SetFileName(file)
    reader.ReadImageInformation()
    
    d[i] = {
        "folder": folder,
        "fn":  file,
        "sz": reader.GetSize(),
        "px": sitk.GetPixelIDValueAsString(reader.GetPixelID()),
        "sp": tuple(round(x,2) for x in reader.GetSpacing()),
        "dir": tuple(int(round(x,1)) for x in reader.GetDirection())
    }
    
d = DF(d)
d

,folder,fn,sz,px,sp,dir
0,ABIDE,../../../mnt/d/PitMRdata/ABIDE/ABIDE/50972/MP-...,"(128, 256, 256)",16-bit signed integer,"(1.33, 1.0, 1.0)","(1, 0, 0, 0, -1, 0, 0, 0, 1)"
1,ABIDE,../../../mnt/d/PitMRdata/ABIDE/ABIDE/51359/MP-...,"(144, 240, 256)",16-bit signed integer,"(1.0, 1.0, 1.0)","(1, 0, 0, 0, -1, 0, 0, 0, 1)"
2,ABIDE,../../../mnt/d/PitMRdata/ABIDE/ABIDE/51313/Mat...,"(128, 128, 34)",16-bit signed integer,"(1.5, 1.5, 4.0)","(1, 0, 0, 0, -1, 0, 0, 0, 1)"
3,ABVIB,../../../mnt/d/PitMRdata/ABVIB/ABVIB/60006/MPR...,"(256, 256, 1)",16-bit unsigned integer,"(1.0, 1.0, 1.0)","(0, 0, -1, 1, 0, 0, 0, -1, 0)"
4,ABVIB,../../../mnt/d/PitMRdata/ABVIB/ABVIB/7002/T1_S...,"(256, 256, 1)",16-bit signed integer,"(1.0, 1.0, 1.0)","(0, 0, -1, 1, 0, 0, 0, -1, 0)"
5,ABVIB,../../../mnt/d/PitMRdata/ABVIB/ABVIB/251/MPRAG...,"(256, 256, 1)",16-bit unsigned integer,"(1.0, 1.0, 1.0)","(0, 0, -1, 0, 0, 0, 0, 0, 0)"
6,ACRIN-FMISO-Brain,../../../mnt/d/PitMRdata/ACRIN-FMISO-Brain/ACR...,"(256, 256, 1)",16-bit signed integer,"(0.94, 0.94, 5.0)","(1, 0, 0, 0, 1, 0, 0, 0, 1)"
7,ACRIN-FMISO-Brain,../../../mnt/d/PitMRdata/ACRIN-FMISO-Brain/ACR...,"(512, 512, 1)",64-bit float,"(0.44, 0.44, 5.0)","(1, 0, 0, 0, 1, 0, 0, 0, 1)"
8,ACRIN-FMISO-Brain,../../../mnt/d/PitMRdata/ACRIN-FMISO-Brain/ACR...,"(512, 512, 1)",16-bit signed integer,"(0.49, 0.49, 3.0)","(0, 0, -1, 1, 0, 0, 0, -1, 0)"
9,ADNI,../../../mnt/d/PitMRdata/ADNI/ADNI1_Complete_1...,"(256, 256, 166)",32-bit float,"(0.94, 0.94, 1.2)","(0, 0, 1, 0, 1, 0, -1, 0, 0)"


In [111]:
def get_img(path):
    # get folder name = data src
    folder = get_folder_name(path)
    
    # get file ext (nii, dcm, etc)
    idx    = src2idx[folder]
    ext    = get_ext(path)
    
    # get file
    # ASSUMES only 1 nii in folder
    
    if ext == ".nii":
        file = os.listdir(str(path))[0]
        file = f"{path}/{file}"
        return sitk.ReadImage(file, sitk.sitkFloat32)

    elif ext == ".dcm":
        reader = sitk.ImageSeriesReader()
        dicom_names = reader.GetGDCMSeriesFileNames(str(path))
        reader.SetFileNames(dicom_names)
        return reader.Execute()

    else:
        print(f"Weird ext - {ext}.")